在深度学习中，数据通常以多维数组（也称为张量）的形式表示。张量的维度通常对应于数据的某种特性。例如，在图像处理中，一个常见的张量格式是（样本数，高度，宽度，通道数），其中样本数是数据集中图像的数量，高度和宽度是图像的像素大小，通道数是颜色通道的数量（例如，RGB图像有3个通道）。

对于时间序列数据或自然语言处理，常见的张量格式是（样本数，序列长度，特征数）。样本数是数据集中序列的数量，序列长度是每个序列的时间步长或词数，特征数是每个时间步长或词的特征数量。

例如，如果你有一个包含100个序列的数据集，每个序列有10个时间步长，并且每个时间步长有8个特征，那么你的输入数据张量的形状将是（100，10，8）。

在准备数据集时，你需要将原始数据转换为这种张量格式，以便可以将其输入到深度学习模型中。这通常涉及到一些预处理步骤，如缩放或标准化特征，填充或截断序列以确保它们的长度相同，以及（对于监督学习任务）将目标变量转换为正确的格式。

In [3]:
import pandas as pd
import numpy as np
import torch

In [4]:
# 打印 PyTorch 版本信息
print(torch.__version__)

# 打印 CUDA 版本
print(torch.version.cuda)

# 打印 cuDNN 版本
print(torch.backends.cudnn.version())

# 检查 GPU 是否可用
print("GPU available:", torch.cuda.is_available())


1.13.1+cu116
11.6
8302
GPU available: True


# 时间序列数据

In [6]:
%pip install openpyxl

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
     --------- ----------------------------- 61.4/250.0 kB 1.7 MB/s eta 0:00:01
     ----------------------------- -------- 194.6/250.0 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 250.0/250.0 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
# 生成数据
# 假设我们有10天的数据，每天24个小时，每个小时有3个特征：'Temperature', 'Humidity', 'WindSpeed'
# 我们可以使用numpy的random函数生成这些数据
data = np.random.rand(10*24, 3)

# 创建数据框
df = pd.DataFrame(data, columns=['Temperature', 'Humidity', 'WindSpeed'])

# 保存到Excel文件
df.to_excel('weather_data.xlsx', index=False)

当我们谈论时间序列数据中的"每个时间步长的特征数"时，我们是指在每个特定的时间点，我们有多少个不同的测量值或观察值。

例如，假设你正在研究天气预报，并且你每小时收集三个数据点：温度、湿度和风速。在这种情况下，你的每个时间步长（每小时）有三个特征。

如果你将这些数据整理成一个形状为（样本数，时间步长，特征数）的张量以供深度学习模型使用，那么"样本数"可能是你收集的总天数，"时间步长"是每天的小时数（例如，24），"特征数"则是3（温度、湿度和风速）。

所以，如果你收集了10天的数据，你的张量形状将是（10，24，3）。这意味着你有10个样本（每个样本代表一天的数据），每个样本有24个时间步长（每个时间步长代表一小时的数据），每个时间步长有3个特征（温度、湿度和风速）。

In [11]:
# 读取Excel文件
df = pd.read_excel('weather_data.xlsx')

# 将数据框转换为numpy数组
data = df.values

# 假设你有10天的数据，每天24个小时，每个小时有3个特征
# 你需要将数据重塑为（样本数，时间步长，特征数）的形状
data = data.reshape((10, 24, 3))

# 将numpy数组转换为PyTorch张量
tensor = torch.from_numpy(data)

print(tensor)

在PyTorch中，通常我们会创建一个`Dataset`对象，然后使用`DataLoader`来批量加载和洗牌数据。以下是如何将你的数据转换为`Dataset`和`DataLoader`的步骤：

1. 创建一个自定义`Dataset`。这需要定义一个类，该类继承自`torch.utils.data.Dataset`，并实现`__len__`和`__getitem__`方法。

2. 使用你的数据创建一个`Dataset`实例。

3. 创建一个`DataLoader`，将你的`Dataset`作为输入。

以下是一个简单的例子：



In [ ]:
from torch.utils.data import Dataset, DataLoader

# 读取Excel文件
df = pd.read_excel('weather_data.xlsx')

# 将数据框转换为numpy数组
data = df.values

# 假设你有10天的数据，每天24个小时，每个小时有3个特征
# 你需要将数据重塑为（样本数，时间步长，特征数）的形状
data = data.reshape((10, 24, 3))

# 创建自定义Dataset
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = torch.from_numpy(data)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

# 创建Dataset实例
dataset = MyDataset(data)

# 创建DataLoader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 使用DataLoader
for batch in dataloader:
    print(batch)



在这个例子中，我们首先创建了一个自定义`Dataset`，它只是简单地返回数据的每个元素。然后，我们使用我们的数据创建了一个`Dataset`实例，并创建了一个`DataLoader`，它每次返回2个样本，并在每个epoch开始时随机洗牌数据。